In [2]:
import numpy as np
import os
import datasets as mt_datasets
import transforms as mt_transforms

import torch
from torchvision import transforms
from torch.utils.data import DataLoader
from torch import autograd, optim
import torch.backends.cudnn as cudnn
import torch.nn as nn

# import torchvision.utils as vutils
# cudnn.benchmark = True
import matplotlib.pyplot as plt
%matplotlib inline

from collections import defaultdict
from pathlib import Path
import imageio

from torchsummary import summary

%load_ext autoreload
%autoreload 2

In [3]:
ROOT_DIR = "/scratch/dj1311/Lung_cancer/LIDC_conversion5/"

In [4]:
img_list = []
label_list = []

for idx, (dirpath, dirnames, filenames) in enumerate(os.walk(ROOT_DIR)):
    if not dirnames:
        # for 1 patient, all files
        nii = []
        flag = False
        for f in filenames:
            if f.startswith("LIDC-IDRI") and f.endswith(".nii.gz"):
                img = f
                nii.append(f)
                    
            if flag == False and f.startswith("Nodule 1 - Annotation") and f.endswith(".nii.gz"):
                gt = f
                nii.append(f)
                flag = True
            
        if len(nii) == 2:
            img_list.append(os.path.join(dirpath, img))
            label_list.append(os.path.join(dirpath, gt))

In [5]:
img_list, label_list = img_list[:10], label_list[:10]
print(len(img_list))
filename_pairs = list(zip(img_list, label_list))

10


In [6]:
train_dataset = mt_datasets.MRI2DSegmentationDataset(filename_pairs, transform=mt_transforms.ToTensor())

In [7]:
patient.keys()


NameError: name 'patient' is not defined

In [8]:
print(patient['filename'])

NameError: name 'patient' is not defined

In [26]:
print(patient['gt_filename'])

/scratch/dj1311/Lung_cancer/LIDC_conversion5/LIDC-IDRI-0289/1.3.6.1.4.1.14519.5.2.1.6279.6001.317889411189319524523491450294/1.3.6.1.4.1.14519.5.2.1.6279.6001.219618492426142913407827034169/Nodule 1 - Annotation Nodule 001.nrrd.nii.gz


In [ ]:
t = patient['gt_filename'].split("nrrd.nii.gz")[0][:-3]
print(t)

In [31]:

s = patient['filename'].split("LIDC-IDRI-")[1][0:4]
print(s)

0289


In [32]:
C = 25
nodules = []
for i, patient in enumerate(train_dataset):
    gt = patient['gt']
    inp = patient['input']

    depth = inp.shape[0]

    idx = gt.nonzero()
    min_, max_ =  idx.min(dim=0)[0], idx.max(dim=0)[0]
    start, end = (min_ +  max_)/2 - C, (min_ +  max_)/2 + C
    
    s_x, s_y, s_z = start.tolist()
    e_x, e_y, e_z = end.tolist()

    if s_x < 0:
        e_x += abs(s_x)
        s_x = 0

    if e_x > depth:
        s_x -= (e_x - depth)
        e_x = depth


    if s_y < 0:
        e_y += abs(s_y)
        s_y = 0

    if e_y > 512:
        s_y -= (e_y - depth)
        e_y = depth


    if s_z < 0:
        e_z += abs(s_z)
        s_z = 0

    if e_z > 512:
        s_z -= (e_z - depth)
        e_z = depth


    nodule = (gt * inp)[s_x:e_x, s_y:e_y, s_z:e_z]
    print(i, patient.id, nodule.shape)
    nodules.append(nodule.unsqueeze(0))


AttributeError: 'dict' object has no attribute 'id'